This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ZKC19940412/water_ice_nep/colab-examples/example-6.ipynb)

In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Hardware accelerator` > `GPU`.

# Install TDMDpy from Source ($\sim$ 2 min)

In [ ]:
%cd /content/
## TODO: Will need to make this "git clone"-able
! unzip tdmdpy-main.zip
! mv tdmdpy-main tdmdpy

# Install tdmdpy from source
%cd tdmdpy
! pip install .

# Install [GPUMD](https://github.com/brucefan1983/GPUMD) from Source ($\sim$ 2 min)
- More instructions can be found : https://gpumd.org/installation.html

In [ ]:
%cd /content/
! git clone https://github.com/brucefan1983/GPUMD.git
%cd /content/GPUMD/src/
! make -j 8
! echo "GPUMD installation finishes!"
%cd /content/

# Clean up Workspace and Fetch Nessary Files

In [ ]:
%cd /content/
## TODO: Will need to make this "git clone"-able
! unzip water_ice_nep-main.zip
! mv water_ice_nep-main water_ice_nep
! rm -r *.zip
! rm -r sample_data/

# Custom Function Define

In [1]:
def set_fig_properties(ax_list):
    tl = 6
    tw = 2
    tlm = 4

    for ax in ax_list:
        ax.tick_params(which='major', length=tl, width=tw)
        ax.tick_params(which='minor', length=tlm, width=tw)
        ax.tick_params(which='both', axis='both', 
                       direction='in', right=True, top=True,
                       left=True)

# Import Necessary Packages

In [2]:
import mdtraj as mdt
import numpy as np
from pynvml import *
from pylab import *  
import scipy.constants as spc 
from tdmdpy.atom_manipulate import decompose_dump_xyz
from tdmdpy.atom_manipulate import load_with_cell
from tdmdpy.create_systems import generate_water_box
from tdmdpy.scorer import score_property
from tdmdpy.thermodynamic_properties import get_block_average_quantities
from tdmdpy.thermodynamic_properties import process_diffusion_coefficients

# Obtain Information about GPU Architecture for Particular colab Instance

In [ ]:
if __name__ == "__main__":

    # Initialize nvml object
    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()

    # Loop through all avaliable devices
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))

# Perform Simulations to Compute  Self Diffusion Coefficients ($ln\mathscr{D}$)

## Create Liquid Water in Cubic Box

In [4]:
if __name__ == "__main__":
    generate_water_box(target_density = 0.994,
                       number_of_molecules=512,
                       is_pre_equilibrate=False)

## Compose run.in

In [ ]:
%%writefile run.in
# Set up NEP potential
potential /content/water_ice_nep/nep-pre-train-model/nep.txt

# time step for 0.5 fs
time_step 0.5


# Initialize velocity at 298K
velocity 298

# Run NPT equalibration with SCR method for 298K Tini and Tend, 
# and 100 for Tcoupling, 0 bar for pressures, 
# and 2 Gpa for pressure coupling and 1000 steps

ensemble npt_scr 298 298 100 0 0 0 2 2 2 1000

# dump extended xyz with every 1000 steps, dump force and velocity too
dump_exyz 1000 1 1

# dump themodynamic quantities every 1000 steps
dump_thermo 1000

# run 20000 steps, equal to 10 ps simulation
run 20000

# Run NVE production run
ensemble nve

# Compute self diffusion coefficnets 
compute_sdc 2 2000

# Run 100000 steps, equal to 50 ps simulation
run 100000

## Peform Simualtions ($\sim$ 15 min)
- 20000 and 100000 steps used here only serve as illustration purpose.

In [ ]:
! /content/GPUMD/src/gpumd < run.in

# Compute Self Diffusion Coefficients ($ln\mathscr{D}$)

In [ ]:
if __name__ == "__main__":
    
    # Process diffusion coefficients from sdc.out
    D, _, _, _, _, _= process_diffusion_coefficients('sdc.out', 
                                                     dimension_factor=3, 
                                                     is_verbose=False)

    # Score property accordingly
    score_property(np.round(np.log(D.mean() * 1e-5),2),  
                   property_str ='lnD_298K')
    print('Stand deviation: %.2f' % np.std(np.log(D* 1e-5)))